<a href="https://colab.research.google.com/github/natanrajch/DiploDatos/blob/main/MELI/MELI_NLP_whoosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gzip
import pandas as pd
import urllib
import tarfile
import urllib.request
import numpy as np
import random
import json
import bisect
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
url_item_data = "https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz"
url_train_data = "https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz"
url_test_data = "https://meli-data-challenge.s3.amazonaws.com/2020/test_dataset.jl.gz"
domain_top_words_url = 'https://raw.githubusercontent.com/natanrajch/DiploDatos/main/MELI/domain_top_words.csv'

In [ ]:
train_data = []
with urllib.request.urlopen(url_train_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    train_data.append(json.loads(line.strip().decode('utf-8'))) 

In [ ]:
train_df = pd.DataFrame(train_data)
del train_data
del gz
train_df

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737
2,"[{'event_info': 248595, 'event_timestamp': '20...",1909110
3,"[{'event_info': 'RADIOBOSS', 'event_timestamp'...",1197370
4,"[{'event_info': 'AMAZFIT BIP', 'event_timestam...",2049207
...,...,...
413158,"[{'event_info': 'PC GAMER', 'event_timestamp':...",459697
413159,"[{'event_info': 289961, 'event_timestamp': '20...",1845503
413160,"[{'event_info': 'ALUGUEL BOB CAT', 'event_time...",2022477
413161,"[{'event_info': 'XAOMI', 'event_timestamp': '2...",1111021


In [ ]:
domain_top_words_df = pd.read_csv(domain_top_words_url, names=['domain_id','top_words'])
domain_top_words_df = domain_top_words_df[~domain_top_words_df.domain_id.isna()]
domain_top_words_df

,domain_id,top_words
1,MLB-CARS_AND_VANS,4x4 manual 8v volkswagen chevrolet aut 5p 16v ...
2,MLB-SNEAKERS,mizuno converse casual olympikus nike sapatêni...
3,MLB-TV_REPLACEMENT_BACKLIGHT_LED_STRIPS,32la613b 39lb5600 39ln5400 39ln5700 42lb5600 3...
4,MLB-AUTOMOTIVE_TAIL_LIGHTS,lanternas fitam circuito tricolor sinaleira bi...
5,MLB-AUTOMOTIVE_DEGREASERS,j50 concentrados multilimpador sintra detersid...
...,...,...
7889,MLM-PERCUSSION_WASHERS,kit led tenis negro envio original gratis bate...
7890,MLM-HARDWARE_NUTS,led tenis negro envio original gratis grado he...
7891,MLM-DIESEL_VACUUM_PUMPS,led tenis negro envio original gratis bomba fr...
7892,MLM-TELEPHONE_BILLING_METERS,funda kit led tenis negro envio original grati...


##ruta1

In [ ]:
!pip install git+https://github.com/stevennic/whoosh.git@FloatingPointCorrections
import os
import shutil
# shutil.rmtree('indexdir')
os.mkdir('indexdir')

  Cloning https://github.com/stevennic/whoosh.git (to revision FloatingPointCorrections) to /tmp/pip-req-build-x9sxttl_
  Running command git clone -q https://github.com/stevennic/whoosh.git /tmp/pip-req-build-x9sxttl_
  Running command git checkout -b FloatingPointCorrections --track origin/FloatingPointCorrections
  Switched to a new branch 'FloatingPointCorrections'
  Branch 'FloatingPointCorrections' set up to track remote branch 'FloatingPointCorrections' from 'origin'.
  Created wheel for Whoosh: filename=Whoosh-2.7.4-py2.py3-none-any.whl size=467501 sha256=8840b8a339a55a49fd4061d4f8119f991ce1fbeea73ca903bc9802848269fc1d
  Stored in directory: /tmp/pip-ephem-wheel-cache-7xndqg5x/wheels/7b/c7/3a/5004ad9757872d23b8ec0d732289f0a1288a17bc177f74cb7f
Successfully built Whoosh


FileExistsError: ignored

In [ ]:
from whoosh.index import create_in
from whoosh.fields import *
schema = Schema(domain_id=TEXT(stored=True), content=TEXT)
ix = create_in("indexdir", schema)
writer = ix.writer()
for row in domain_top_words_df.itertuples(index=False):
  writer.add_document(domain_id=row.domain_id, content=row.top_words)
writer.commit()

# from whoosh import qparser
# with ix.searcher() as searcher:
#      query = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup).parse("reloj^2 samsung") #se agregar el parámetro group que activará OR entre las palabras
#      print(query)
#      results = searcher.search(query, terms=True)
#      print('got this far')
#      if results:
#       for result in results:
#         print(result.values()[0], item_df.title[result.values()[0]])

In [ ]:
#test de funcionamiento
test = "mouse inalambrico"
from whoosh import qparser
with ix.searcher() as searcher:
     query = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup).parse(test) #se agregar el parámetro group que activará OR entre las palabras
     print(query)
     results = searcher.search(query, terms=True)
     print('got this far')
     if results:
      for result in results:
        print(result.values()[0])

(content:mouse OR content:inalambrico)
got this far
MLM-COMPUTER_MICE
MLM-KEYBOARD_AND_MOUSE_KITS
MLB-COMPUTER_MICE
MLB-KEYBOARD_AND_MOUSE_KITS
MLB-COMPUTER_REMOTE_CONTROLS
MLM-KARAOKE_MICROPHONES
MLM-CELLPHONE_REPLACEMENT_KEYBOARDS
MLB-MOUSE_PADS
MLM-MOUSE_PADS
MLM-BOX_SPRING_COVERS


Debo crear el item_domain_dict para poder hacer análisis sobre los domain de cada compra

In [ ]:
item_data = []
with urllib.request.urlopen(url_item_data) as handle:
  gz = gzip.GzipFile(fileobj=handle)
  for i, line in enumerate(gz):
    item_data.append(json.loads(line.strip().decode('utf-8'))) 

In [ ]:
item_df = pd.DataFrame(item_data)
del item_data
del gz

In [ ]:
item_list = item_df['item_id'].tolist()
domain_list = item_df['domain_id'].tolist()
item_domain_dict = dict(zip(item_list, domain_list))

In [ ]:
from whoosh import qparser
query_object = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup) #se agregar el parámetro group que activará OR entre las palabras
def count_bdom_in_searches(row, max_events=10,query_object=query_object):
  ''' Cuenta cantidad de veces que aparece el domain de la compra entre las searches. 
  Considera únicamente los últimos max_events cantidad de eventos, para no explotar la PC'''
  counter = 0
  bdom = item_domain_dict[row.item_bought]
  query_string = ''
  max_search = 3
  search_count = 0
  for event in row.user_history[:max_events]: #Se limita la cantidad de eventos a buscar
    if search_count < max_search:  
      if event['event_type'] == 'search':
        search_count += 1
        query_string += ' ' + event['event_info'] #se unifica todas las search como 1 solo string por motivos de rendimiento
  with ix.searcher() as searcher:
        #query_string = ' '.join(event['event_info'].split(' ')[:1]) tomando los primeros n terminos
        #query_string = event['event_info'].split(' ')[0] tomando solo el primer termino
    query = query_object.parse(query_string)  #Esto se podría quizás mejorar elevando al ^2 el primer término buscado
    results = searcher.search(query)  
    if results:
      result = results[0].values()[0] #Se toma únicamente el elemento más relevante de la búsqueda
      if result == bdom:
        counter += 1
  return counter
          

In [ ]:
' '.join('hola que'.split()[:4])

'hola que'

In [ ]:
tqdm.pandas()
train_df['counter_bdom_in_searches'] = train_df.progress_apply(count_bdom_in_searches,axis=1)
train_df[6700:6703]

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


,user_history,item_bought,counter_bdom_in_searches
6700,"[{'event_info': 1471860, 'event_timestamp': '2...",1135771,0
6701,"[{'event_info': 919566, 'event_timestamp': '20...",1811871,0
6702,"[{'event_info': 'TORO', 'event_timestamp': '20...",865535,0


In [ ]:
train_df.counter_bdom_in_searches.to_csv('counter_bdom_in_searches.csv', index=False)

In [ ]:
train_df['domain_was_viewed'] = train_df.progress_apply(lambda row: item_domain_dict[row['item_bought']] in [item_domain_dict[history['event_info']] for history in row['user_history'] if history['event_type'] == 'view'], axis=1)


In [ ]:
train_df['bdom_in_events'] = train_df.domain_was_viewed + train_df.counter_bdom_in_searches
train_df.bdom_in_events.value_counts() 

0    205459
1    162598
2     45106
Name: bdom_in_events, dtype: int64

In [ ]:
(45106+162598), 205459

(207704, 205459)

In [ ]:
'se identifican solo nuevos: ', sum((train_df.domain_was_viewed == False) & (train_df.counter_bdom_in_searches > 0))

('se identifican solo nuevos: ', 3847)

In [ ]:
import os
os.mkdir('indexdir2')

FileExistsError: ignored

In [ ]:
domain_top_words_5_url = 'https://raw.githubusercontent.com/natanrajch/DiploDatos/main/MELI/domain_top_words_5.csv'
domain_top_words_df_5 = pd.read_csv(domain_top_words_5_url, names=['domain_id','top_words'])
domain_top_words_df_5 = domain_top_words_df_5[~domain_top_words_df_5.domain_id.isna()]
domain_top_words_df_5

,domain_id,top_words
0,domain_id,top_words
1,MLB-CARS_AND_VANS,4x4 manual 8v volkswagen chevrolet aut 16v gas...
2,MLB-SNEAKERS,mizuno converse casual olympikus nike sapatêni...
3,MLB-TV_REPLACEMENT_BACKLIGHT_LED_STRIPS,32la613b 39lb5600 39ln5400 39ln5700 42lb5600 3...
4,MLB-AUTOMOTIVE_TAIL_LIGHTS,lanternas fitam circuito tricolor sinaleira bi...
...,...,...
7889,MLM-PERCUSSION_WASHERS,kit led tenis negro envio original tenis adida...
7890,MLM-HARDWARE_NUTS,led tenis negro envio original gratis tenis ad...
7891,MLM-DIESEL_VACUUM_PUMPS,led tenis negro envio original gratis 100 orig...
7892,MLM-TELEPHONE_BILLING_METERS,funda kit led tenis negro envio tenis adidas 1...


In [ ]:
from whoosh.index import create_in
from whoosh.fields import *
schema = Schema(domain_id=TEXT(stored=True), content=TEXT)
ix = create_in("indexdir2", schema)
writer = ix.writer()
for row in domain_top_words_df_5.itertuples(index=False):
  writer.add_document(domain_id=row.domain_id, content=row.top_words)
writer.commit()

In [ ]:
#test de funcionamiento
test = "silla living"
from whoosh import qparser
with ix.searcher() as searcher:
     query = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup).parse(test) #se agregar el parámetro group que activará OR entre las palabras
     print(query)
     results = searcher.search(query, terms=True)
     if results:
      for result in results:
        print(result.values()[0])

(content:silla OR content:living)
MLM-STYLING_CHAIRS
MLM-CHAIR_COVERS
MLM-BABY_HIGH_CHAIRS
MLM-BABY_CAR_SEATS
MLM-OFFICE_CHAIRS
MLM-GARDEN_CHAIRS
MLM-DINING_CHAIRS
MLM-BABY_BOUNCERS
MLM-BICYCLE_CHILD_SEATS
MLM-WHEELCHAIR_CUSHIONS


In [ ]:
from whoosh import qparser
query_object = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup) #se agregar el parámetro group que activará OR entre las palabras
def count_bdom_in_searches(row, max_events=10,query_object=query_object):
  ''' Cuenta cantidad de veces que aparece el domain de la compra entre las searches. 
  Considera únicamente los últimos max_events cantidad de eventos, para no explotar la PC'''
  counter = 0
  bdom = item_domain_dict[row.item_bought]
  query_string = ''
  max_search = 3
  search_count = 0
  with ix.searcher() as searcher:
    for event in row.user_history[:max_events]: #Se limita la cantidad de eventos a buscar
      if event['event_type'] == 'search':
        #query_string = ' '.join(event['event_info'].split(' ')[:1]) #tomando los primeros n terminos
        query_string = event['event_info'].split(' ')[0] #tomando solo el primer termino
        query = query_object.parse(query_string)  #Esto se podría quizás mejorar elevando al ^2 el primer término buscado
        results = searcher.search(query)  
        if results:
          result = results[0].values()[0] #Se toma únicamente el elemento más relevante de la búsqueda
          if result == bdom:
            counter += 1
  return counter

In [ ]:
tqdm.pandas()
train_df['counter_bdom_in_searches_5'] = train_df.progress_apply(count_bdom_in_searches,axis=1)
train_df[6700:6703]

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


KeyboardInterrupt: ignored

##Sección trabajo anterior

In [ ]:
# os.mkdir('indexdir2')

In [ ]:
dfm.title[0:10]

0    Casa Sola En Venta Con Gran Patio Solo Pago De...
1    Resident Evil Origins Collection Nintendo Swit...
2                        Falda De Imitación Piel Negra
3         Powercolor Red Devil Radeon Rx 580 8gb Gddr5
4    Laptop Hp Nx6320 Core Duo Con Puerto Db9 Windo...
5    Transmisor Fm Sin Hilos Bluetooth Reproductor ...
6       Funda Cartera Caseme 007 2 En 1 Huawei P30 Pro
7    Lampara De Techo Tubo Slim Led Base G5 9w Blan...
8    Repisa Organizador Para Cocina,  Especieros, T...
9    Trío, Balerina, Cosmo, Leopardo/negro/rojo, Ve...
Name: title, dtype: object

KeyboardInterrupt: ignored

In [ ]:
with ix.searcher() as searcher:
     query = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup).parse("muñeca barbie") #se agregar el parámetro group que activará OR entre las palabras
     print(query)
     results = searcher.search(query, terms=True)
     print('got this far')
     if results:
      for result in results:
        print(result.values()[0], item_df.title[result.values()[0]])

Debo crear un feature que indique si una fila pertenecerá a MLM oa MLB. Para los que tengan views, será el domain del MLM que corresponda. Para los que sólo posean searches, se debe utilizar un predictor de lenguaje entre español y portugués. Si está en inglés, se asociará a MLB ya que es lo más común.

In [ ]:
train_df.user_history[1][1]

{'event_info': 'DESMAMADEIRA ELETRICA',
 'event_timestamp': '2019-10-07T09:45:29.322-0400',
 'event_type': 'search'}

In [ ]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor



In [ ]:
from polyglot.detect import Detector

mixed_text = train_df.user_history[1][1]['event_info']
Detector(mixed_text).languages[0].name
        # print(language)

Detector is not able to detect the language reliably.


'Portuguese'

In [ ]:
item_list_condition = item_df['item_id'].tolist()
domain_list = item_df['domain_id'].tolist()
item_domain_dict = dict(zip(item_list_condition, domain_list))


NameError: ignored

In [ ]:
def find_country(row):
  for event in row.user_history:
    if event['event_type'] == 'view':
      domain = item_domain_dict[event['event_info']]
      if domain:
        return item_domain_dict[event['event_info']][:3]
  for event in row.user_history:
    try:
      lang = Detector(event['event_info'], quiet=True).languages[0].name
      if lang == 'Portuguese':
        return 'MLB'
      elif lang == 'Spanish':
        return 'MLM'
    except:
      pass
  return 'MLB'

In [ ]:
train_df['country'] = train_df.apply(find_country, axis=1)

In [ ]:
train_df.country.value_counts()
55763/(55763+357400)

0.13496610296662576

In [ ]:
os.mkdir('indexdir3')
os.mkdir('indexdir4')

Crear índices de item_data en portugués y en español, para hallar resultados relevantes de búsquedas

In [ ]:
from whoosh.index import create_in
from whoosh.fields import *
schema = Schema(df_index=NUMERIC(stored=True), content=TEXT)
ix_MLB = create_in("indexdir3", schema)
writer_MLB = ix_MLB.writer()
for index, title in dfb['title'].items():
  writer_MLB.add_document(df_index=index, content=title)
writer_MLB.commit()

ix_MLM = create_in("indexdir4", schema)
writer_MLM = ix_MLM.writer()
for index, title in dfm['title'].items():
  writer_MLM.add_document(df_index=index, content=title)
writer_MLM.commit()


# from whoosh import qparser
# with ix.searcher() as searcher:
#      query = qparser.QueryParser("content", ix.schema, group=qparser.OrGroup).parse("reloj^2 samsung") #se agregar el parámetro group que activará OR entre las palabras
#      print(query)
#      results = searcher.search(query, terms=True)
#      print('got this far')
#      if results:
#       for result in results:
#         print(result.values()[0], item_df.title[result.values()[0]])
        

In [ ]:
dfm['title'].items()

In [ ]:
with ix_MLB.searcher() as searcher:
          query = query_object.parse('MODER DIET')  #Esto se podría quizás mejorar elevando al ^2 el primer término buscado
          results = searcher.search(query)
          print(results)
          if not results:
            print('funca como false')
          else:
            print('funca como tru')
          result = results[0].values()[0] #Se toma únicamente el elemento más relevante de la búsqueda
          print(result)
          for result in results:
            print(result)

<Top 10 Results for Or([Term('content', 'moder'), Term('content', 'diet')]) runtime=0.005306553000991698>
funca como tru
1642700
<Hit {'df_index': 1642700}>
<Hit {'df_index': 755543}>
<Hit {'df_index': 566118}>
<Hit {'df_index': 741441}>
<Hit {'df_index': 829350}>
<Hit {'df_index': 850820}>
<Hit {'df_index': 1106368}>
<Hit {'df_index': 1195858}>
<Hit {'df_index': 1219776}>
<Hit {'df_index': 1500695}>


In [ ]:
item_df[1642700:1642701]

,item_id,title,domain_id,product_id,price,category_id,condition
1642700,429310,Moder Diet,MLB-SUPPLEMENTS,None,600.00,MLB264201,new


In [ ]:
item_index = item_df.index.tolist()
# domain_list = item_df['domain_id'].tolist()
index_domain_dict = dict(zip(item_list_condition, domain_list))


In [ ]:
from whoosh import qparser
query_object = qparser.QueryParser("content", ix_MLM.schema, group=qparser.OrGroup) #se agregar el parámetro group que activará OR entre las palabras
def count_bdom_in_searches(row, max_events=20,query_object=query_object):
  ''' Cuenta cantidad de veces que aparece el domain de la compra entre las searches. 
  Considera únicamente los últimos max_events cantidad de eventos, para no explotar la PC'''
  counter = 0
  bdom = item_domain_dict[row.item_bought]
  for event in row.user_history[:max_events]: #Se limita la cantidad de eventos a buscar
    if event['event_type'] == 'search':
      if row.country == 'MLB':
        with ix_MLB.searcher() as searcher:
          query = query_object.parse(event['event_info'])  #Esto se podría quizás mejorar elevando al ^2 el primer término buscado
          results = searcher.search(query)  #Se toma únicamente el elemento más relevante de la búsqueda
          if results:
            result = results[0].values()[0]
            if index_domain_dict[result] == bdom:
              counter += 1
      else:
        with ix_MLM.searcher() as searcher:
          query = query_object.parse(event['event_info'])  #Esto se podría quizás mejorar elevando al ^2 el primer término buscado
          results = searcher.search(query) #Se toma únicamente el elemento más relevante de la búsqueda
          if results:
            result = results[0].values()[0]
            if index_domain_dict[result] == bdom:
              counter += 1
  return counter
          

In [ ]:
test_search = train_df[:50].apply(count_bdom_in_searches,axis=1)

In [ ]:
train_df.user_history[24:25].values

array([list([{'event_info': 830693, 'event_timestamp': '2019-10-19T12:33:35.362-0400', 'event_type': 'view'}, {'event_info': 1722160, 'event_timestamp': '2019-10-19T14:29:44.368-0400', 'event_type': 'view'}, {'event_info': 'MODER DIET', 'event_timestamp': '2019-10-19T14:30:36.206-0400', 'event_type': 'search'}, {'event_info': 243621, 'event_timestamp': '2019-10-19T14:31:06.982-0400', 'event_type': 'view'}, {'event_info': 'MODER DIET', 'event_timestamp': '2019-10-19T14:32:48.121-0400', 'event_type': 'search'}, {'event_info': 1421378, 'event_timestamp': '2019-10-19T14:33:26.848-0400', 'event_type': 'view'}, {'event_info': 'DESINCHA', 'event_timestamp': '2019-10-19T14:33:43.876-0400', 'event_type': 'search'}, {'event_info': 144569, 'event_timestamp': '2019-10-19T14:34:37.678-0400', 'event_type': 'view'}, {'event_info': 'BANDEIRAS ACEITAS', 'event_timestamp': '2019-10-21T10:15:58.855-0400', 'event_type': 'search'}, {'event_info': 'MODER DIET', 'event_timestamp': '2019-10-21T12:14:51.124-04